In [1]:
from keras import backend as K
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import time
import os

np.random.seed(1337)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras import backend as K
from numpy import nan
import sys
import datetime

now = datetime.datetime.now

In [3]:
from matplotlib import pyplot as plt

%matplotlib inline

In [4]:
# dimensions of the generated pictures for each filter.
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [5]:
img_rows, img_cols = IMG_WIDTH, IMG_HEIGHT

if K.image_data_format() == 'channels_first':
    shape_ord = (3, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)


In [6]:
shape_ord

(224, 224, 3)

In [7]:
from keras.applications import vgg16
from keras.layers import Input

# build the VGG16 network with ImageNet weights
vgg16 = vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input((224, 224, 3)))
print('Model loaded.')

Model loaded.


In [8]:
vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
now = datetime.datetime.now

batch_size = 32
nb_classes = 4
nb_epoch = 50

# input image dimensions
img_rows, img_cols = 224, 224
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

In [10]:
# input_shape = (img_rows, img_cols,3)
# classification_layers = [
#     Flatten(),
#     Dense(128),
#     Activation('relu'),
#     Dropout(0.5),
#     Dense(128),
#     Activation('relu'),
#     Dropout(0.5),
#     Dense(nb_classes),
#     Activation('softmax')
# ]

In [11]:
# from keras.models import Model
# x = vgg16.output


In [12]:
for l in vgg16.layers:
    l.trainable = False

In [13]:
#model = Sequential(vgg16 + classification_layers)

In [14]:
from keras.models import Model
#model = Model(inputs=vgg16.input,outputs=vgg16.output)

In [71]:
# model = Sequential()
# for l in vgg16.layers:
#     l.trainable = False
#     model.add(l)
# model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [15]:
from keras.layers.normalization import BatchNormalization

x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
x = Dense(4096, activation='relu', name='ft_fc1')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(4, activation = 'softmax')(x)

In [16]:
#create graph of your new model
model = Model(inputs=vgg16.input, outputs=predictions)

#compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
# model.add()
# model.add(Dense(768, activation='relu',input_shape=(512,)))
# model.add(Dropout(0.5))
# model.add(Dense(768, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(4, activation='softmax'))

In [47]:
# out_list = [(None, 224, 224, 3),(None, 224, 224, 64),(None, 224, 224, 64),(None, 112, 112, 64),
#  (None, 112, 112, 128),(None, 112, 112, 128),(None, 56, 56, 128),(None, 56, 56, 256),
#  (None, 28, 28, 256),(None, 28, 28, 512),(None, 28, 28, 512),(None, 14, 14, 512),
#  (None, 14, 14, 512),(None, 14, 14, 512),(None, 14, 14, 512),(None, 7, 7, 512),
#  (None, 25088),(None, 768), (None,768),(None,4)]

In [48]:

# for layer, o in zip(model.layers, out_list):
#     layer.Input = o
#     print(layer.Output)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [77]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
t = now()

In [18]:
train_datagen = ImageDataGenerator(
        zoom_range=0.2,
        horizontal_flip=True)

In [19]:
train_generator = train_datagen.flow_from_directory(
        '../data/train',
        target_size=(224, 224),
        batch_size=32, #changed from 1, change back if needed
        class_mode='categorical')

Found 554 images belonging to 4 classes.


In [20]:
test_datagen = ImageDataGenerator()

In [21]:
validation_generator = test_datagen.flow_from_directory(
        '../data/validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical')

Found 188 images belonging to 4 classes.


In [84]:
model.fit_generator(
        train_generator,
        steps_per_epoch=24,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=6)

Epoch 1/50
24/24 [==============================] - 22s - loss: 0.5421 - acc: 0.8244 - val_loss: 1.7895 - val_acc: 0.6809
Epoch 2/50
24/24 [==============================] - 19s - loss: 0.3125 - acc: 0.8980 - val_loss: 1.6619 - val_acc: 0.6755
Epoch 3/50
24/24 [==============================] - 19s - loss: 0.1708 - acc: 0.9399 - val_loss: 1.2496 - val_acc: 0.7021
Epoch 4/50
24/24 [==============================] - 19s - loss: 0.1547 - acc: 0.9461 - val_loss: 1.3583 - val_acc: 0.6543
Epoch 5/50
24/24 [==============================] - 19s - loss: 0.0856 - acc: 0.9687 - val_loss: 1.2928 - val_acc: 0.6862
Epoch 6/50
24/24 [==============================] - 19s - loss: 0.0857 - acc: 0.9751 - val_loss: 1.3173 - val_acc: 0.6968
Epoch 7/50
24/24 [==============================] - 19s - loss: 0.0524 - acc: 0.9830 - val_loss: 1.3116 - val_acc: 0.6755
Epoch 8/50
24/24 [==============================] - 20s - loss: 0.0269 - acc: 0.9935 - val_loss: 1.4013 - val_acc: 0.6489
Epoch 9/50
24/24 [======

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________